---
title: "Tutoriel de différentiation automatique"
from: markdown+emoji
lang: fr
format: html
toc: true
author:
  - name: Mahendra Mariadassou
    orcid: 0000-0003-2986-354X
    email: mahendra.mariadassou@inrae.fr
    affiliations:
      - name: INRAE - MaIAGE
        adress: Domaine de Vilvert
        city: Jouy-en-Josas
        state: France
  - name: Hugo Gangloff
  - name: Arthur Leroy
  - name: Lucia Clarotto
date: "2025-08-18"
date-modified: today
date-format: "[Last Updated on] MMMM, YYYY"
---


::: callout-note
L'obectif de ce tutoriel est de montrer comment utiliser JAX et PyTorch pour calculer le JVP et le VJP d'une fonction simple:
$$
f: x \in \mathbb{R}^p \mapsto tanh(a^\top x + b) \in \mathbb{R}
$$
avec $a \in \mathbb{R}^p$ et $b \in \mathbb{R}$. 
On considère les dérivées par rapport à l'entrée $x$ puis par rapport aux paramètres $a, b$. 
:::

# Théorie [XX]

# Exemple en Torch [MM, LC]

::: panel-tabset

## Dérivée par rapport à $x$

## Dérivée par rapport à $a$ et $b$

:::

# Exemple en JAX [HG, AL]

Pour illustrer la différentiation automatique, nous allons utiliser JAX, sur une function simple dont nous connaissons les gradients analytiques.


In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit
from jax import random

dim = 100

a = jnp.arange(dim)/ (dim*10)
b = 0.5
x = (jnp.arange(dim) - 37) / (dim*10)

def f(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

f(x, a, b) 

::: panel-tabset

## Dérivée par rapport à $x$

On définit la fonction les gradients exacts à partir des formules analytiques. 


In [ ]:
def df_dx(x, a, b):
    return a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)

Puis on définit les gradients via autograd et on vérifie que les résultats sont identiques.


In [ ]:
## jax.grad calcule la formule backward par défaut
grad_df_dx = jax.grad(lambda x: f(x, a, b), argnums=0) 
fwdgrad_df_dx = jax.jacfwd(lambda x: f(x, a, b), argnums=0) 

assert jnp.allclose(grad_df_dx(x), df_dx(x, a, b))
assert jnp.allclose(grad_df_dx(x), fwdgrad_df_dx(x))

print('All good, we are ready to go!')

## Dérivée par rapport à $a$ et $b$

On définit la fonction les gradients exacts à partir des formules analytiques. 


In [ ]:
def df_dab(x, a, b):
    return x * (1 - jnp.tanh(jnp.dot(a, x) + b) **2), 1 - jnp.tanh(jnp.dot(a, x) + b) **2

Puis on définit les gradients via autograd et on vérifie que les résultats sont identiques.


In [ ]:
## jax.grad calcule la formule backward par défaut
grad_df_dab = jax.grad(lambda a_b: f(x, *a_b), argnums=0)
fwdgrad_df_dab = jax.jacfwd(lambda a_b: f(x, *a_b), argnums=0)

assert all(jnp.allclose(grad_df_dab((a,b))[i], df_dab(x, a, b)[i]) for i in range(2))
assert all(jnp.allclose(grad_df_dab((a,b))[i], fwdgrad_df_dab((a,b))[i]) for i in range(2))

print('All good, we are ready to go!')

::: 

Nous avons donc bien vérifié que les gradients calculés avec JAX sont identiques aux gradients analytiques.

# Comparaison des temps de calcul en JAX entre autograd (backward et forward) et le calcul explicite des gradients

Pour mettre en lumière les différences entre backward et forward, nous allons définir une nouvelle fonction dont les sorties sont de plus grandes dimensions que les entrées. 

::: panel-tabset

## $x \in \mathbb{R}^d$, f(x) $\in \mathbb{R}$

Comparaison des temps de calcul :

In [ ]:
%timeit df_dx(x, a, b).block_until_ready()
%timeit grad_df_dx(x).block_until_ready()
%timeit fwdgrad_df_dx(x).block_until_ready()

## $x \in \mathbb{R}^d$, f(x) $\in \mathbb{R}$
On définit une nouvelle fonction $g$ multidimensionnelle:


In [ ]:
dim_g = 100

def g(x):
    return jnp.array([jnp.tanh(x + i/dim_g) for i in range(dim_g)])

def dg_dx(x):
    return jnp.array([1- jnp.tanh(x + i/dim_g)**2 for i in range(dim_g)])
    
grad_dg_dx = jax.jacrev(lambda x: g(x), argnums=0)
fwdgrad_dg_dx = jax.jacfwd(lambda x: g(x), argnums=0)    

x_g = 0.5

Comparaison des temps de calcul :

In [ ]:
%timeit dg_dx(x_g).block_until_ready()
%timeit grad_dg_dx(x_g).block_until_ready()
%timeit fwdgrad_dg_dx(x_g).block_until_ready()

:::

On onbserve que le calcul explicite des gradients est plus rapide que les deux autres méthodes. Il y a donc un prix à payer pour la différentiation automatique, bien qu'elle est très utile pour les fonctions complexes. Cependant nous allons voir qu'en combinaison avec le jitting, les choses changent.

# JIT compilation avec JAX 

La compilation JIT (Just-In-Time) permet d'optimiser les performances des fonctions en les compilant à la volée. JAX fournit la fonction `jit` pour cela.

::: panel-tabset

## Gradients jittés pour la fonction $f$

In [ ]:
@jit
def f_jit(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

@jit
def df_dx(x, a, b):
    return a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)

df_dx_jit = jax.jit(df_dx)
grad_f_jit = jax.jit(jax.grad(lambda x: f_jit(x, a, b), argnums=0))
fwdgrad_f_jit = jax.jit(jax.jacfwd(lambda x: f_jit(x, a, b), argnums=0))

%timeit df_dx_jit(x, a, b).block_until_ready()
%timeit grad_f_jit(x).block_until_ready()
%timeit fwdgrad_f_jit(x).block_until_ready()

### Gradients jittés pour $g$

## Gradients jittés pour la fonction $g$

In [ ]:
@jit
def g_jit(x):
    return jnp.array([jnp.tanh(x + i/dim_g) for i in range(dim_g)])
    
@jit
def dg_dx_jit(x):
    return jnp.array([1- jnp.tanh(x + i/dim_g)**2 for i in range(dim_g)])

dg_dx_jit = jax.jit(dg_dx)
grad_g_jit = jax.jit(jax.jacrev(lambda x: g_jit(x), argnums=0))
fwdgrad_g_jit = jax.jit(jax.jacfwd(lambda x: g_jit(x), argnums=0))

%timeit dg_dx_jit(x).block_until_ready()
%timeit grad_g_jit(x).block_until_ready()
%timeit fwdgrad_g_jit(x).block_until_ready()

:::